# Import packages

In [50]:
!pip install quandl

In [0]:
import quandl
import pandas as pd

In [0]:
quandl.ApiConfig.api_key = 'txyy5YsPXKcYkozp7vYs'

# Get the dataset from multiple sources

## Stocks

In [0]:
GM_stocks = quandl.get('EOD/GM')

In [0]:
GM_stocks.index.names = ['date']

## Facebook (FBD)

In [0]:
GM_FBD = quandl.get_table('SMA/FBD', brand_ticker='GM', paginate=True)

In [0]:
GM_FBD = GM_FBD.groupby('date').sum()

## Facebook (FBP)

In [0]:
GM_FBP = quandl.get_table('SMA/FBP', brand_ticker='GM', paginate=True)

In [0]:
GM_FBP['created_time'] = pd.to_datetime(GM_FBP['created_time']).dt.strftime('%Y-%m-%d')

In [0]:
GM_FBP = GM_FBP.replace(['POSITIVE', 'NEGATIVE', 'NEUTRAL', 'UNDEFINED'], [1, -1, 0, 0])

In [0]:
GM_FBP = GM_FBP.groupby('date').sum()

## Twitter(TWTD)

In [0]:
GM_TWTD = quandl.get_table('SMA/TWTD', brand_ticker='GM', paginate=True)

In [0]:
GM_TWTD = GM_TWTD.groupby('date').sum()

In [0]:
oil = pd.read_csv('wti-crude-oil-prices-10-year-daily-chart.csv')

In [0]:
oil['date']= pd.to_datetime(oil['date'])

In [0]:
oil = oil.set_index('date')

## Merge

In [0]:
result = pd.merge(GM_stocks, GM_FBP, on='date')

In [0]:
result = pd.merge(result, GM_FBD, on='date')

In [0]:
result = pd.merge(result, GM_TWTD, on='date')

In [0]:
result = pd.merge(result, oil, on='date')

In [0]:
result = result.drop(columns=['Open',	'High',	'Low',	'Volume',	'Dividend', 'Split',	'Adj_Open',	'Adj_High',	'Adj_Low',	'Adj_Close',	'Adj_Volume'])

## Feature generation

In [0]:
temps = pd.DataFrame(result.Close)

In [0]:
width = 5
shifted = temps.shift(width - 1)
window = shifted.rolling(window=width)
rw_week = pd.concat([window.min(), window.mean(), window.max(), window.std()], axis=1)
rw_week.columns = ['min_wk', 'mean_wk', 'max_wk', 'std_wk']

In [0]:
width = 22
shifted = temps.shift(width - 1)
window = shifted.rolling(window=width)
rw_month = pd.concat([window.min(), window.mean(), window.max(), window.std()], axis=1)
rw_month.columns = ['min_mth', 'mean_mth', 'max_mth', 'std_mth']

In [0]:
result = pd.concat([result, rw_week, rw_month], axis=1)

In [0]:
result = result.dropna()

In [0]:
df = result
business_days = pd.bdate_range(start=df.index[0], end=df.index[-1])

In [0]:
business = pd.DataFrame(business_days, columns=['date'])

In [0]:
result = pd.merge(business, result, on='date', how='outer')

In [0]:
result = result.interpolate()

In [0]:
result = result.dropna()

In [0]:
result.to_csv('dataset.csv')

## Downsample

In [0]:
result['date'] = pd.to_datetime(result['date']).dt.strftime('%Y-%m')

In [0]:
result_ds = result.groupby('date').sum()

In [0]:
result_ds.to_csv('dataset_ds.csv')